In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


from input_generator import load_dataset, load_dataset_with_lables
from data_parser import get_tags_and_labels, get_vocab, get_tags

d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
# Get dataframe with tags and category_id columns
csvpath = 'data/CAvideos.cvs'
tab_dataframe = get_tags_and_labels(csvpath)

# Creating different coloumn based on different tags
new = tab_dataframe["tags"].str.split(".", expand = True)

max_of_tags = 15

for i in range(0, max_of_tags):
    name = "tags"+str(i)
    tab_dataframe[name] = new[i]

# Dropping old tags columns 
tab_dataframe.drop(columns =["tags"], inplace = True) 
tab_dataframe.fillna("notags", inplace = True) 


tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,tags9,tags10,tags11,tags12,tags13,tags14
0,10,Eminem,Walk,On,Water,Aftermath/Shady/Interscope,Rap,notags,notags,notags,notags,notags,notags,notags,notags,notags
1,23,plush,bad unboxing,unboxing,fan mail,idubbbztv,idubbbztv2,things,best,packages,plushies,chontent chop,notags,notags,notags,notags
2,23,racist superman,rudy,mancuso,king,bach,racist,superman,love,rudy mancuso poo bear black white official mus...,iphone x by pineapple,lelepons,hannahstocking,rudymancuso,inanna,anwar
3,24,ryan,higa,higatv,nigahiga,i dare you,idy,rhpc,dares,no truth,comments,comedy,funny,stupid,fail,notags
4,10,edsheeran,ed sheeran,acoustic,live,cover,official,remix,official video,lyrics,session,notags,notags,notags,notags,notags


In [3]:
# Getting leaning vocab
vocab = get_vocab(get_tags(csvpath))


In [4]:
# Mapping tags to indexes in vocab
for index in range(0, max_of_tags):
    tab_dataframe['tags{}'.format(index)] = tab_dataframe['tags{}'.format(index)].map(voc_di)
    
tab_dataframe.fillna(0.0, inplace = True) 
tab_dataframe['tags0'].unique
# tab_dataframe.head()

<bound method Series.unique of 0         97865.0
1         56319.0
2        102394.0
3         86917.0
4        125837.0
           ...   
40876      2914.0
40877    103965.0
40878     47551.0
40879     37622.0
40880     50086.0
Name: tags0, Length: 40881, dtype: float64>

In [5]:
from sklearn import preprocessing
df = tab_dataframe.copy()
x = df.values
standard_scaler = preprocessing.StandardScaler()
x_scaled = standard_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns=tab_dataframe.columns)
tab_dataframe = df

tab_dataframe.head()

,category_id,tags0,tags1,tags2,tags3,tags4,tags5,tags6,tags7,tags8,tags9,tags10,tags11,tags12,tags13,tags14
0,-1.593446,0.390095,0.038237,-1.354903,-1.029633,-1.102045,-0.351190,-1.285006,-1.210916,-1.190013,-1.122252,-1.084996,-1.037027,-0.994109,-0.950567,-0.917268
1,0.325381,-0.548924,0.841871,1.442897,0.053094,-0.492575,-1.250101,1.786166,0.124584,1.251466,0.075815,0.767155,-1.037027,-0.994109,-0.950567,-0.917268
2,0.325381,0.492459,1.658030,-0.380591,0.223491,-0.991603,1.438828,1.201242,1.680526,-0.402790,0.557300,1.620424,1.130516,-0.570936,1.839734,1.050000
3,0.472983,0.142649,-0.813318,-0.842974,-0.501740,-0.761887,1.082209,0.986434,-0.060438,1.522101,0.877793,1.619337,1.820434,1.949531,0.730366,-0.917268
4,-1.593446,1.022316,-0.393781,-0.832569,0.312358,-1.216356,-1.019818,1.125975,-0.881581,0.506680,-0.646453,-1.084996,-1.037027,-0.994109,-0.950567,-0.917268


In [6]:
# Create train,validation and test examples from dataframe
# train_ds, val_ds, test_ds = load_dataset(tab_dataframe)
(train_ds, train_lb), (val_ds, val_lb), (test_ds, test_lb) = load_dataset_with_lables(tab_dataframe, 'category_id')


In [7]:
# first 5 training examples
print(train_ds.values[5:10])
train_lb.values[0:5]

[[ 0.54161807  1.06345585 -0.22506441  1.19241336 -0.74821889  0.39446741
  -1.27098538  0.17042037  1.11995212  0.027279    0.13217428 -0.78737269
   1.95444381 -0.95013596  1.66840667]
 [ 0.43545688 -0.67241037  1.02304572 -0.2936935  -1.03218823  0.16673761
  -1.2538146   0.71085846  1.78026535  0.50725186  0.1797928   1.19478344
   0.16460487 -0.44323089  0.15628976]
 [ 1.38900901  0.94692401  0.9996712   0.00834028  0.47105763 -0.37258884
  -1.09905402  0.96803699 -0.51738928  1.60848941 -0.00412013 -1.03702658
  -0.99410893 -0.95056728 -0.91726833]
 [ 0.07156608  0.30362021  0.62815839  1.73653919  1.69163792 -1.33442131
  -1.28500649 -1.21091649 -1.19001319 -1.12225226 -1.084996   -1.03702658
  -0.99410893 -0.95056728 -0.91726833]
 [ 0.25640453  1.17045129  1.47858387  1.74251203  1.09230364 -0.41520193
  -0.59179454  1.57715978 -0.38891775 -0.01794972  1.05989001 -1.01967842
  -0.14185032 -0.63640574 -0.44727041]]


array([-1.59344644,  0.47298316,  0.47298316, -2.92186546, -0.1174253 ])

In [8]:
hidden = 32
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(max_of_tags,)),
    layers.Dense(hidden,activation='relu'),
    layers.Dropout(0.7),
    layers.Dense(2*hidden,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.7),
    layers.Dense(44, activation='softmax', kernel_regularizer=keras.regularizers.l2(0.01))])

model.compile(optimizer='nadam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', 'sparse_categorical_crossentropy'],
              )

history = model.fit(train_ds.values, train_lb.values,
                    batch_size=64,
                    epochs=100,
                    validation_data=(val_ds, val_lb))


W0810 11:08:08.815292 10672 nn_ops.py:4220] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0810 11:08:08.899336 10672 nn_ops.py:4220] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0810 11:08:09.118706 10672 deprecation.py:323] From d:\miniconda3\envs\deep\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 26163 samples, validate on 6541 samples
Epoch 1/100


InvalidArgumentError:  Received a label value of -2 which is outside the valid range of [0, 44).  Label values: 0 0 0 0 0 0 0 0 0 0 -2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -1 0 0 -1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 -1 0 0 0 0 0 0 0 0
	 [[node metrics/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-8-2af090a5e365>:19) ]] [Op:__inference_keras_scratch_graph_2287]

Function call stack:
keras_scratch_graph
